# WattTime
This Python module was created to facilitate interaction with the WattTime API.
This module provides full API functionality can be accessed through this module, as well as some additional tools that make 
- https://www.watttime.org/api-documentation

## Register Account
#### Skip this step if you already have an account.
- https://www.watttime.org/api-documentation/#register-new-user

In [ ]:
from WattTime import WattTime

username = "{USERNAME}"
password = "{PASSWORD}"
email = "{EMAIL_ADDRESS}"
org = "{ORG_NAME}"

wt = WattTime.RegisterNewUser(username, password, email)


## Instantiate Class and Authenticate
#### Requires account username and password

In [ ]:
from WattTime import WattTime
import os


username = os.getenv("WATTTIME_API_USERNAME")
password = os.getenv("WATTTIME_API_PASSWORD")

wt = WattTime.GridEmissionsInformation(username, password)

## Grid Emissions Information (Class Methods)

### Determine Grid Region
- https://www.watttime.org/api-documentation/#determine-grid-region

In [ ]:
latitude = 33.844978
longitude = -118.387238
wt.determine_grid_region(latitude, longitude)

### List of Grid Regions
- https://www.watttime.org/api-documentation/#list-of-grid-regions

In [ ]:
all_regions = False

wt.list_grid_regions(all_regions)

### Real-time Emissions Index
- https://www.watttime.org/api-documentation/#real-time-emissions-index

#### Search by Balancing Authority Abbreviation

In [ ]:
bal_auth_abbr = "CAISO_NORTH"
wt.real_time_emissions_index(bal_auth_abbr=bal_auth_abbr)

#### Search by Latitude and Longitude

In [ ]:
latitude = 33.844978
longitude = -118.387238
lati_long = (latitude, longitude)
wt.real_time_emissions_index(lati_long=lati_long, style="moer")

### Grid Emissions Data
- https://www.watttime.org/api-documentation/#grid-emissions-data

#### Search by Balancing Authority Abbreviation

In [ ]:
bal_auth_abbr = "CAISO_NORTH"
wt.grid_emissions_data(bal_auth_abbr=bal_auth_abbr)

#### Search by Latitude and Longitude

In [ ]:
latitude = 33.844978
longitude = -118.387238
lati_long = (latitude, longitude)
wt.grid_emissions_data(lati_long=lati_long)

### Historical Emissions

In [ ]:
bal_auth_abbr="CAISO_NORTH"
filename = "historical_emissions"
extract_files = True
concatenate = True

wt.historical_emissions(bal_auth_abbr, filename)


### Emissions Forecast

In [ ]:
bal_auth_abbr="CAISO_NORTH"

data = wt.emissions_forcast(bal_auth_abbr, extended_forecast=False)
print(f"Data generated at: {data.get('generated_at')}. \n{len(data.get('forecast'))} datapoints found")

### Grid Region Map Geometry

In [ ]:
from WattTime import WattTime
import os


username = os.getenv("WATTTIME_API_USERNAME")
password = os.getenv("WATTTIME_API_PASSWORD")

wt = WattTime.GridEmissionsInformation(username, password)

In [ ]:
import time


timestamp_list = []
for i in range(5):
    timestamp_list.append(int(time.time()) + i)


timestamp_list.reverse()  # reverse so that order is oldest to newest

print(timestamp_list)
print("\n")



for i in range(6, 11):
    timestamp_list = timestamp_list[1:] + [int(time.time()) + (i-5)]  # remove first item in list and replace last item with newest timestamp
    print(timestamp_list)



### Rate Limiting Algorithm



rate_limit = r (requests)
rate_time = t (seconds)


1. New request comes in.
2. create new cur_timestamp
3. (list comprehension) filter timestamp_list  for timestamps newer than (cur_timestamp - rate limit period)
4. If list comprehension list length...
	1. is gte than rate limit calls, calculate seconds between first item in list and cur_timestamp, and use that amount for time.sleep
	2. is less than rate limit calls, process new request:
		1. remove first item from list and add new timestamp to end of list
		2. 


In [ ]:
import requests
import time
import warnings


class RateLimitTest:
    """Class to assist with rate limiting API requests

        Args:
            calls (int): Maximum number of API calls within time 'period'.
            period (int): Amount of seconds before the rate limit resets.

        Returns:
            dict: Returns the details of the balancing authority (BA) serving that location, if known,
                or a Coordinates not found error if the point lies outside of known/covered BAs.
        """
    def __init__(self, calls: int = 60, period: int = 60, cushion: int = 0):
        self.calls = calls
        self.period = period
        self.cushion = cushion
        self._timestamp_list = [time.time() - period] * calls
        self._session = requests.Session()

    def _check_rate_limit(self):
        ts_now = time.time()
        request_count = len([ts for ts in self._timestamp_list if ts > (ts_now - self.period)])
        if request_count >= self.calls:
            rate_limit_cooldown = int(ts_now - self._timestamp_list[0]) + self.cushion
            # warnings.warn(f"API rate limit reached, waiting for reset in {rate_limit_cooldown} seconds")
            time.sleep(rate_limit_cooldown)
        self._timestamp_list = self._timestamp_list[1:] + [ts_now]
        return

    def make_request(self, counter) -> None:
        self._check_rate_limit()
        webhook = "https://events.hookdeck.com/e/src_OQSYxUeuKD0k"
        data = {"counter": counter}
        try:
            r = self._session.post(webhook, json=data)
            if r.status_code == 429:
                raise Exception(f"HTTPError {r.status_code}: {r.reason}")
            else:
                # Catch all other HTTP errors
                r.raise_for_status()
        except requests.exceptions.HTTPError as e:
            raise



In [ ]:
from tqdm import trange


calls = 60
period = 60
limiter = RateLimitTest(calls, period, cushion=10)

for i in trange(120):
    limiter.make_request(i)
